In [2]:
import sys
sys.path.append("..")
import pandas as pd
import numpy as np
import seaborn as sns
from datasets import *

%matplotlib inline


import warnings
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, recall_score, precision_score, confusion_matrix, mean_squared_error
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.model_selection import KFold, train_test_split


warnings.filterwarnings("ignore")
pd.set_option("display.max_rows", 500)
pd.set_option("display.max_columns", 40)
random_state = 20180415

In [3]:
excel_file = 'graduation_rates_public_school_2010_2011.xlsx'
df1 = pd.read_excel(excel_file,sheet_name=0)

In [4]:
df1[df1['School Name']=='Townsend Harris High School']

,DBN,School Name,Category,Cohort Year,Cohort,Cohort Total,Grads Total,Grads Percent of Cohort,Regents Total,Regents Percent of Cohort,Regents Percent of Grads,Advanced Regents Total,Advanced Regents Percent of Cohort,Advanced Regents Percent of Grads,Regents without Advanced Total,Regent without Advanced Percent of Cohort,Regents without Advanced Percent of Grads,Local Total,Local Percent of Cohort,Local Percent of Grads,Still Enrolled Total,Still Enrolled Percent of Cohort,Dropout Total,Dropout Percent of Cohort
3954,25Q525,Townsend Harris High School,All Students,2011,4 year August,310,310,100,310,100,100,307,99.0323,99.0323,3,0.967742,0.967742,0,0,0,0,0,0,0
3955,25Q525,Townsend Harris High School,All Students,2010,4 year August,282,282,100,282,100,100,277,98.227,98.227,5,1.77305,1.77305,0,0,0,0,0,0,0
3956,25Q525,Townsend Harris High School,All Students,2011,4 year June,310,310,100,310,100,100,307,99.0323,99.0323,3,0.967742,0.967742,0,0,0,0,0,0,0
3957,25Q525,Townsend Harris High School,All Students,2010,4 year June,282,282,100,282,100,100,277,98.227,98.227,5,1.77305,1.77305,0,0,0,0,0,0,0
3958,25Q525,Townsend Harris High School,All Students,2011,5 year August,310,310,100,310,100,100,307,99.0323,99.0323,3,0.967742,0.967742,0,0,0,0,0,0,0
3959,25Q525,Townsend Harris High School,All Students,2010,5 year August,282,282,100,282,100,100,277,98.227,98.227,5,1.77305,1.77305,0,0,0,0,0,0,0
3960,25Q525,Townsend Harris High School,All Students,2011,5 year June,310,310,100,310,100,100,307,99.0323,99.0323,3,0.967742,0.967742,0,0,0,0,0,0,0
3961,25Q525,Townsend Harris High School,All Students,2010,5 year June,282,282,100,282,100,100,277,98.227,98.227,5,1.77305,1.77305,0,0,0,0,0,0,0
3962,25Q525,Townsend Harris High School,All Students,2011,6 year June,310,310,100,310,100,100,307,99.0323,99.0323,3,0.967742,0.967742,0,0,0,0,0,0,0
3963,25Q525,Townsend Harris High School,All Students,2010,6 year June,282,282,100,282,100,100,277,98.227,98.227,5,1.77305,1.77305,0,0,0,0,0,0,0


In [74]:
df2 = pd.read_csv('2019_2010_schools_progress.csv')
classsize = pd.read_csv('class_size.csv')

In [6]:
df2.head()

,DBN,DISTRICT,SCHOOL,PRINCIPAL,PROGRESS REPORT TYPE,SCHOOL LEVEL*,PEER INDEX*,2009-2010 OVERALL GRADE,2009-2010 OVERALL SCORE,2009-2010 ENVIRONMENT CATEGORY SCORE,2009-2010 ENVIRONMENT GRADE,2009-2010 PERFORMANCE CATEGORY SCORE,2009-2010 PERFORMANCE GRADE,2009-2010 PROGRESS CATEGORY SCORE,2009-2010 PROGRESS GRADE,2009-2010 ADDITIONAL CREDIT,2008-09 PROGRESS REPORT GRADE
0,01M015,1,P.S. 015 Roberto Clemente,Thomas Staebell,EMS,Elementary,62.65,C,37.6,7.2,B,3.0,D,27.4,B,0.0,B
1,01M019,1,P.S. 019 Asher Levy,Jacqueline Flanagan,EMS,Elementary,48.94,C,35.7,6.4,B,4.9,D,24.4,B,0.0,A
2,01M020,1,P.S. 020 Anna Silver,James Lee,EMS,Elementary,57.68,A,70.3,9.3,A,7.9,C,44.1,A,9.0,A
3,01M034,1,P.S. 034 Franklin D. Roosevelt,Joyce Stallings Harte,EMS,K-8,66.75,B,53.0,6.8,B,7.4,C,33.0,B,5.8,A
4,01M063,1,P.S. 063 William McKinley,Darlene Despeignes,EMS,Elementary,57.55,B,54.0,7.8,B,7.1,C,35.8,A,3.3,B


In [77]:
df3 = pd.merge(df1, df2, on='DBN')
df3 = pd.merge(df3, classsize, on='DBN')

In [78]:
df3.head()

,DBN,School Name,Category,Cohort Year,Cohort,Cohort Total,Grads Total,Grads Percent of Cohort,Regents Total,Regents Percent of Cohort,Regents Percent of Grads,Advanced Regents Total,Advanced Regents Percent of Cohort,Advanced Regents Percent of Grads,Regents without Advanced Total,Regent without Advanced Percent of Cohort,Regents without Advanced Percent of Grads,Local Total,Local Percent of Cohort,Local Percent of Grads,...,Dropout Percent of Cohort,DISTRICT,SCHOOL,PRINCIPAL,PROGRESS REPORT TYPE,SCHOOL LEVEL*,PEER INDEX*,2009-2010 OVERALL GRADE,2009-2010 OVERALL SCORE,2009-2010 ENVIRONMENT CATEGORY SCORE,2009-2010 ENVIRONMENT GRADE,2009-2010 PERFORMANCE CATEGORY SCORE,2009-2010 PERFORMANCE GRADE,2009-2010 PROGRESS CATEGORY SCORE,2009-2010 PROGRESS GRADE,2009-2010 ADDITIONAL CREDIT,2008-09 PROGRESS REPORT GRADE,min,max,mean
0,01M292,Orchard Collegiate Academy,All Students,2011,4 year August,73,46,63.0137,41,56.1644,89.1304,0,0,0,41,56.1644,89.1304,5,6.84932,10.8696,...,9.58904,1,Henry Street School for International Studies,Erin Balet,HS,High School,2.16,D,40.0,5.0,F,13.3,C,21.7,F,0.0,B,11.0,26.0,18.961538
1,01M292,Orchard Collegiate Academy,All Students,2011,4 year August,73,46,63.0137,41,56.1644,89.1304,0,0,0,41,56.1644,89.1304,5,6.84932,10.8696,...,9.58904,1,Henry Street School for International Studies,Erin Balet,EMS,Middle,2.31,C,33.7,3.2,D,4.6,D,21.9,C,4.0,B,11.0,26.0,18.961538
2,01M292,Orchard Collegiate Academy,All Students,2010,4 year August,61,26,42.623,26,42.623,100,1,1.63934,3.84615,25,40.9836,96.1538,0,0,0,...,27.8689,1,Henry Street School for International Studies,Erin Balet,HS,High School,2.16,D,40.0,5.0,F,13.3,C,21.7,F,0.0,B,11.0,26.0,18.961538
3,01M292,Orchard Collegiate Academy,All Students,2010,4 year August,61,26,42.623,26,42.623,100,1,1.63934,3.84615,25,40.9836,96.1538,0,0,0,...,27.8689,1,Henry Street School for International Studies,Erin Balet,EMS,Middle,2.31,C,33.7,3.2,D,4.6,D,21.9,C,4.0,B,11.0,26.0,18.961538
4,01M292,Orchard Collegiate Academy,All Students,2011,4 year June,73,46,63.0137,41,56.1644,89.1304,0,0,0,41,56.1644,89.1304,5,6.84932,10.8696,...,9.58904,1,Henry Street School for International Studies,Erin Balet,HS,High School,2.16,D,40.0,5.0,F,13.3,C,21.7,F,0.0,B,11.0,26.0,18.961538


In [84]:
cols = ['Regents Percent of Cohort', 'Advanced Regents Percent of Cohort', 'Regent without Advanced Percent of Cohort', 'Dropout Percent of Cohort','mean','max','min']
df3[cols] = df3[cols].apply(pd.to_numeric,errors='coerce')
df3.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 4760 entries, 0 to 4759
Data columns (total 43 columns):
DBN                                          4760 non-null object
School Name                                  4760 non-null object
Category                                     4760 non-null object
Cohort Year                                  4760 non-null int64
Cohort                                       4760 non-null object
Cohort Total                                 4760 non-null int64
Grads Total                                  4760 non-null object
Grads Percent of Cohort                      4760 non-null object
Regents Total                                4760 non-null object
Regents Percent of Cohort                    4740 non-null float64
Regents Percent of Grads                     4760 non-null object
Advanced Regents Total                       4760 non-null object
Advanced Regents Percent of Cohort           4740 non-null float64
Advanced Regents Percent of Grads  

In [85]:
features = (['DISTRICT', 'PEER INDEX*', '2009-2010 ENVIRONMENT CATEGORY SCORE', '2009-2010 PERFORMANCE CATEGORY SCORE', '2009-2010 PROGRESS CATEGORY SCORE','Regents Percent of Cohort', 'Advanced Regents Percent of Cohort', 'Regent without Advanced Percent of Cohort', 'Dropout Percent of Cohort','mean','max','min'])

##features = (['Grads Percent of Cohort'])

model_df = df3[(features + ['Grads Percent of Cohort'])].dropna().reset_index()

train_df, holdout_df, y_train, y_holdout = train_test_split(
    model_df[features], 
    model_df['Grads Percent of Cohort'], test_size=0.1,
    random_state=random_state)

train_df['Grads Percent of Cohort'] = y_train
holdout_df['Grads Percent of Cohort'] = y_holdout

train_df.reset_index(inplace=True)
holdout_df.reset_index(inplace=True)

print(train_df.shape[0], train_df['Grads Percent of Cohort'].mean())
print(holdout_df.shape[0], holdout_df['Grads Percent of Cohort'].mean())





3690 74.41229264232112
410 75.32638559816834


In [86]:
model_df.shape

(4100, 14)

In [87]:
train_df.shape

(3690, 14)

In [88]:
holdout_df.shape
holdout_df

,index,DISTRICT,PEER INDEX*,2009-2010 ENVIRONMENT CATEGORY SCORE,2009-2010 PERFORMANCE CATEGORY SCORE,2009-2010 PROGRESS CATEGORY SCORE,Regents Percent of Cohort,Advanced Regents Percent of Cohort,Regent without Advanced Percent of Cohort,Dropout Percent of Cohort,mean,max,min,Grads Percent of Cohort
0,3147,22,3.27,12.5,17.2,44.5,100.000000,45.098038,54.901962,0.000000,25.570968,34.0,18.0,100
1,2266,13,2.27,8.1,14.6,28.6,12.244898,0.000000,12.244898,38.775509,24.394444,32.0,16.0,14.2857
2,3431,25,2.45,8.5,10.5,29.2,54.304634,12.450331,41.854305,15.231788,24.097727,33.8,5.0,57.351
3,47,1,2.21,11.1,14.7,44.1,88.636360,0.000000,88.636360,7.954545,24.021739,33.0,14.7,90.9091
4,998,6,2.22,11.1,19.8,44.5,67.955803,0.000000,67.955803,16.574585,21.676923,33.0,6.0,75.1381
5,3642,28,2.57,12.4,10.4,32.8,66.143791,13.986928,52.156864,8.235294,27.956757,36.0,12.5,68.4967
6,2646,17,2.35,5.2,14.9,24.5,62.857143,0.000000,62.857143,4.285714,24.636842,32.0,15.5,68.5714
7,2755,17,3.23,11.3,19.2,45.7,98.780487,7.317073,91.463417,1.219512,23.723810,33.5,10.0,98.7805
8,1652,10,1.85,11.3,17.0,38.1,53.424656,15.068493,38.356163,13.698630,28.385714,33.5,24.0,56.1644
9,1618,10,2.58,13.0,25.0,59.3,85.245903,0.000000,85.245903,6.557377,26.696296,33.0,6.0,91.8033


In [89]:
k_fold = KFold(n_splits=5, random_state=random_state)

In [90]:
def get_cv_results(classifier):
    
    results = []
    for train, test in k_fold.split(train_df):
        classifier.fit(train_df.loc[train, features], train_df.loc[train, 'Grads Percent of Cohort'])
        y_predicted = classifier.predict(train_df.loc[test, features])
        accuracy = mean_squared_error(train_df.loc[test,'Grads Percent of Cohort'], y_predicted)
        results.append(accuracy)
        print(classifier.score(train_df.loc[train, features],train_df.loc[train, 'Grads Percent of Cohort']))
    
    return np.mean(results), np.std(results)

In [91]:
reg = LinearRegression()
get_cv_results(reg)


0.9435536695807089
0.9434424689716072
0.9433084781110792
0.9429266257910759
0.9433666255660863


(21.623949832401397, 0.9468176222447461)